Global Parameters


In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo, showerror
from tkinter.scrolledtext import ScrolledText
from tkinter import PhotoImage
from pathlib import Path
from functools import partial
from fileorganizer import FileOrganizer 
import utils as ut
import pandas as pd
import numpy as np
import json
from pprint import pprint
import dataframecleaner as dfc

In [2]:
#colors
main_colour = '#8991DF'
colour2 = '#377FA5'
colour3 = '#4650B3'
colour4 = '#891A99'

In [3]:
#Files dictionary
raw_files ={}
global df_parameters_raw 
global df_antenna_raw
global dict_parameter 


# Welcome

In [4]:
def validate_button():
    global df_parameters_raw 
    global df_antenna_raw
    messages = []

    for key, uploaded_file in raw_files.items():
        # print(keys)
        # print(uploaded_file)
        if key == 'ParametersDAB':
            df_parameters_raw = uploaded_file.file_processer()
            messages.append(uploaded_file.get_message())
        elif key == 'AntennaDAB':
            df_antenna_raw = uploaded_file.file_processer()
            messages.append(uploaded_file.get_message())
        else:
            print('Entered on default')
        #if isinstance(df, pd.DataFrame):
            #print(df)
            
    print(f'The raw files: {raw_files}')


    if not all(key in raw_files for key in ["AntennaDAB", "ParametersDAB"]):
        showerror(title="Error", message="Not all files are present, please check")
    else:
        showinfo(title="Validation", message=f"{messages[0]}\n\n {messages[1]} ")
        if (isinstance(df_parameters_raw, pd.DataFrame)  and  isinstance(df_antenna_raw, pd.DataFrame)):
            continue_button.config(state="normal")
            print(f'The parameters DAB: {df_parameters_raw.head()}')
            print(f'The antenna DAB: {df_antenna_raw.head()}')

In [5]:
def select_file(file_key):
    filetypes = (("CSV Files", "*.csv"), ("JSON", "*.json"))

    filename = fd.askopenfilename(
        title="Open a file",
        initialdir="C:/Users/herms/Formative_Assignment",
        filetypes=filetypes,
    )

    if filename:
        route = Path(filename)
        file_type = ut.check_file_format(route)
        open_file = ut.load_file(route)
        categorized_file = FileOrganizer(file_type,open_file, file_key)
        showinfo(title="File Selected", message=filename)
        raw_files[file_key] = categorized_file

    else:
        showinfo(title="Error", message="File not selected")

In [6]:
def save_antenna_data_asjson(df):
    df.to_json(r'C:\Users\herms\Formative_Assignment\saved_antenna.json', orient ='index')

In [7]:
def save_parameter_data_asjson(df):
    df.to_json(r'C:\Users\herms\Formative_Assignment\saved_parameter.json', orient ='index')

In [8]:
def continue_window():
    global df_parameters_raw
    global df_antenna_raw

    # Call to functions that make the preliminar data cleaning 
    # PARAMETERS DATA
    #Setting id column as index
    df_parameters_raw = dfc.setting_indexes(df_parameters_raw, 'id')

    #Changing Date Type
    #df_parameters_raw['Date'] = pd.to_datetime(df_parameters_raw['Date']).dt.date

    #Changing the Format in  Site
    df_parameters_raw['Site'] = df_parameters_raw['Site'].str.title()


    # ANTENNA DATA
    df_antenna_raw = dfc.setting_indexes(df_antenna_raw, 'id')
    #Setting New Column Order
    new_antenna_columns =['NGR', 'Longitude/Latitude', 'Long', 'Lat', 'Site Height', 
        'In-Use Ae Ht','In-Use ERP Total','Dir Max ERP', 
        '0','10','20','30','40','50','60','70','80','90',
        '100','110','120','130','140','150','160','170','180','190',
        '200','210','220','230','240','250','260','270','280','290',
        '300','310','320','330','340','350']
    
    #Changing the of In-Use ERP Total
    df_antenna_raw = df_antenna_raw[new_antenna_columns]
    df_antenna_raw = dfc.convert_to_float(df_antenna_raw, 'In-Use ERP Total')

    print(f'The parameters DAB\n: {df_parameters_raw.head()}')
    print(f'The antenna DAB\n: {df_antenna_raw.head()}')

    
    save_parameter_data_asjson(df_parameters_raw)
    save_antenna_data_asjson(df_antenna_raw)

    main_window.withdraw()
    define_work_window()

In [9]:
# Creating Main Window
main_window = tk.Tk()
main_window.title("Main Window")
main_window.geometry("900x450")
main_window.resizable(False, False)

# Main Layout
main_window.grid_rowconfigure(0, weight=1)
main_window.grid_columnconfigure(0, weight=2)
main_window.grid_columnconfigure(1, weight=1)

# LEFT FRAME
left_frame = tk.Frame(main_window, bg="white")
left_frame.grid(row=0, column=0, sticky="nsew")

# RIGHT FRAME
right_frame = tk.Frame(main_window, bg=main_colour)
right_frame.grid(row=0, column=1, sticky="nsew")

for i in range(10):
    left_frame.grid_rowconfigure(i, weight=1)
    left_frame.grid_columnconfigure(0, weight=1)

    right_frame.grid_rowconfigure(i, weight=1)
    right_frame.grid_columnconfigure(0, weight=1)
    right_frame.grid_columnconfigure(2, weight=1)

#ELEMENTS OF THE WINDOW
# Image
image_bck = PhotoImage(file="ed-howell-flow-graph_sm.png")
label_im = tk.Label(left_frame, image=image_bck)
label_im.image = image_bck
label_im.grid(row=3, column=0)

# Copyright Text
label_text = tk.Label(
    left_frame,
    text="This is a product developed by Student S.A. All rights reserved",
    fg="black"
)
label_text.grid(row=6, column=0)

# CSV Files
# File1
# Text
file1_text_label = tk.Label(
    right_frame, text="Parameters File", bg=main_colour, fg="white"
)
file1_text_label.grid(row=3, column=1)

# Button
open_file1_button = tk.Button(
    right_frame, text="Select Parameters File", width=16, command=partial(select_file, "ParametersDAB")
)
open_file1_button.grid(row=3, column=2)

# File2
# Text
file2_text_label = tk.Label(
    right_frame, text="Antenna File", bg=main_colour, fg="white"
)
file2_text_label.grid(row=4, column=1)
# Button
open_file2_button = tk.Button(
    right_frame, text="Select Antenna File", width=16, command=partial(select_file, "AntennaDAB")
)
open_file2_button.grid(row=4, column=2)

# Validate Files Button
validate_button = tk.Button(
    right_frame,
    text="Validate Files",
    command=validate_button,
    background="white",
    foreground="black",
    activebackground="lightgrey",
    width=15,
    height=1,
    border=1,
    font=("Arial", 9),
)
validate_button.grid(row=5, column=2)

# Continue Button
continue_button = tk.Button(
    right_frame,
    text="Continue",
    command=continue_window,
    background="white",
    foreground="black",
    activebackground="lightgrey",
    width=13,
    height=1,
    border=1,
    font=("Arial", 9, "bold"),
    state ='disabled'
)
continue_button.grid(row=8, column=2)

# main_window.update_idletasks()

# Work Window

In [10]:
def display_table(frame, df, column_name):
    sliced_df = df.loc[:,:column_name]
    tree = ttk.Treeview(frame, columns=list(sliced_df.columns), show="headings")

    for column in sliced_df.columns:
        tree.heading(column, text=column)
        tree.column(column, width=100)

    tree.grid(row=0, column=0, sticky="nsew")

    for index, row in sliced_df.iterrows():
        tree.insert("", tk.END, values=list(row))

    tree.grid_rowconfigure(0, weight=1)
    tree.grid_columnconfigure(0, weight=1)

In [11]:
def search_by_value(df, column_name, value):
    return df.loc[df[column_name] == value]

In [12]:
def close_all_windows():
    main_window.destroy()

In [13]:
def define_work_window():
    global df_parameters
    global df_antenna
    global NGR_list

    df_parameters = pd.read_json('saved_parameter.json', orient ='index')
    df_antenna = pd.read_json('saved_antenna.json', orient ='index')

    def drop_NGRs():
        global df_parameters
        global df_antenna
        global NGR_list

        indexes_to_drop = df_antenna[df_antenna['NGR'].isin(NGR_list)].index
        df_parameters = df_parameters.drop(indexes_to_drop)
        df_antenna = df_antenna.drop(indexes_to_drop)
        
        print("NGR dropped")
        showinfo(title="Info", message="NGRs dropped")
        print(df_antenna.index)

    def add_NGR(cNGR_get_text, cNGR_list_print):
        global NGR_list
        global df_antenna

        NGR = cNGR_get_text.get()
        NGR_exists = df_antenna['NGR'].isin([NGR])
        
        if any(NGR_exists):
            NGR_list.append(NGR)
            cNGR_list_print = update_print_list(cNGR_list_print)
        else:
            showinfo(title="Error", message="NGR not present")

        print(NGR_list)

    def update_print_list(cNGR_list_print):    
        cNGR_list_print.delete('1.0', tk.END)
        for item in NGR_list:
            cNGR_list_print.insert(tk.END, item)
            cNGR_list_print.insert(tk.END, '\n')
        cNGR_list_print.update()
        return cNGR_list_print

    work_window = tk.Tk()
    work_window.title("Work Window")
    work_window_width = work_window.winfo_screenwidth()
    work_window_height = work_window.winfo_screenheight()
    work_window.geometry("%dx%d" % (work_window_width, work_window_height))
    work_window.resizable(True, True)

    # Main Layout
    work_window.grid_rowconfigure(0, weight=1)
    work_window.grid_columnconfigure(0, weight=6)
    work_window.grid_columnconfigure(1, weight=5)

    # LEFT FRAME
    left_frame = tk.Frame(work_window, bg="white")
    left_frame.grid(row=0, column=0, sticky="nsew")

    # RIGHT FRAME
    right_frame = tk.Frame(work_window, bg=colour2)
    right_frame.grid(row=0, column=1, sticky="nsew")

    left_frame.grid_columnconfigure(0, weight=1)
    right_frame.grid_columnconfigure(0, weight=1)
    right_frame.grid_columnconfigure(1, weight=2)
    right_frame.grid_columnconfigure(2, weight=1)

    for i in range(10):
        left_frame.grid_rowconfigure(i, weight=1)
        right_frame.grid_rowconfigure(i, weight=1)


    # WINDOW ELEMENTS
    # Left Elements
    # Table 1
    table1_frame = tk.Frame(left_frame, bg=colour3)
    display_table(table1_frame, df_parameters, "EID")
    table1_frame.grid(row=1, column=0)

    # Table 2
    table2_frame = tk.Frame(left_frame, bg=colour3)
    display_table(table2_frame, df_antenna, "Dir Max ERP")
    table2_frame.grid(row=4, column=0)

    save_parameter_file_button = tk.Button(
        left_frame,
        text="Save Parameter File",
        height=2,
        border=1,
        width=15,
        command=partial(save_parameter_data_asjson, df_parameters),
        background = main_colour,
        foreground = 'white'
    )
    save_parameter_file_button.grid(row=2, column=0)

    save_antenna_file_button = tk.Button(
        left_frame,
        text="Save Antenna File",
        height=2,
        border=1,
        width=15,
        command=partial(save_antenna_data_asjson, df_antenna),
        background = main_colour,
        foreground = 'white'
    )
    save_antenna_file_button.grid(row=5, column=0)

    #RIGHT ELEMENTS
    #Clean_by_NGR_Frame
    NGR_list = ['NZ03586532','NJ94270253','SP820141','NN11277352']
    clean_NGR_frame = tk.Frame(right_frame, bg = colour2, highlightthickness=2, highlightbackground="white")
    clean_NGR_frame.grid(row = 1, column=1, sticky="nsew", padx=10)

    #configure internal grid
    for i in range(6):
        clean_NGR_frame.grid_rowconfigure(i, weight=1)
    clean_NGR_frame.grid_columnconfigure(0, weight=2, )
    clean_NGR_frame.grid_columnconfigure(1, weight=1)

    #internal Left
    cNGR_title_label = tk.Label(clean_NGR_frame, text = 'NGR EXCLUSION', bg = colour2, fg ='white')
    cNGR_title_label.grid(row = 0, column=0)

    cNGR_inst_label = tk.Label(clean_NGR_frame, text = 'Enter NGR to drop', width=15,  bg = colour2, fg ='white')
    cNGR_inst_label.grid(row = 1, column = 0)

    cNGR_get_text = tk.Entry(clean_NGR_frame)
    cNGR_get_text.grid(row = 2, column = 0)

    cNGR_list_print = ScrolledText(clean_NGR_frame, width=15, height=4) # todo check 
    cNGR_list_print.grid(row = 2, column = 1, rowspan=2)
    cNGR_list_print = update_print_list(cNGR_list_print)

    cNGR_add_button = tk.Button(clean_NGR_frame, text="Add NGR",command=partial(add_NGR, cNGR_get_text, cNGR_list_print))
    cNGR_add_button.grid(row = 3, column = 0)

    #Internal Right

    cNGR_inst_label = tk.Label(clean_NGR_frame, text = 'List of dropped NGRs',  bg = colour2, fg ='white')
    cNGR_inst_label.grid(row = 1, column = 1) 

    cNGR_drop_button = tk.Button(clean_NGR_frame, text = 'Drop NGRs', command = drop_NGRs)
    cNGR_drop_button.grid(row = 4, column = 1)

    work_window.protocol("WM_DELETE_WINDOW", close_all_windows)

In [14]:
main_window.mainloop()

File is CSV
File is CSV
El archivo es True
El archivo es True
The raw files: {'ParametersDAB': <fileorganizer.FileOrganizer object at 0x000002A52D165690>, 'AntennaDAB': <fileorganizer.FileOrganizer object at 0x000002A52BB57710>}
The parameters DAB:        id        Date          Ensemble     Licence Ensemble Area   EID  \
0  745395  24/05/2016          Aberdeen    DL000024      Aberdeen  C19A   
1  745402  11/09/2008  BBC National DAB  BBCD000001            UK  CE15   
2  745418  18/01/2017  BBC National DAB  BBCD000001            UK  CE15   
3  745451  22/10/2014  BBC National DAB  BBCD000001            UK  CE15   
4  745498  03/12/2009  BBC National DAB  BBCD000001            UK  CE15   

  Transmitter Area           Site    Freq. Block  ... Data Serv Label11  \
0          Redmoss        Redmoss  220.352   11C  ...               NaN   
1      Abergavenny    Abergavenny  225.648   12B  ...               NaN   
2        Aylesbury  Aylesbury ATE  225.648   12B  ...               NaN   


c:\Users\herms\Formative_Assignment\dataframecleaner.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col_name] = df[col_name].str.replace('.', '')
C:\Users\herms\AppData\Local\Temp\ipykernel_3664\1261579927.py:6: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_parameters = pd.read_json('saved_parameter.json', orient ='index')


The parameters DAB
:               Date          Ensemble     Licence Ensemble Area   EID  \
id                                                                     
745395  24/05/2016          Aberdeen    DL000024      Aberdeen  C19A   
745402  11/09/2008  BBC National DAB  BBCD000001            UK  CE15   
745418  18/01/2017  BBC National DAB  BBCD000001            UK  CE15   
745451  22/10/2014  BBC National DAB  BBCD000001            UK  CE15   
745498  03/12/2009  BBC National DAB  BBCD000001            UK  CE15   

       Transmitter Area           Site    Freq. Block TII Main Id (Hex)  ...  \
id                                                                       ...   
745395          Redmoss        Redmoss  220.352   11C                3E  ...   
745402      Abergavenny    Abergavenny  225.648   12B                24  ...   
745418        Aylesbury  Aylesbury Ate  225.648   12B                16  ...   
745451       Brecon VHF     Brecon Vhf  225.648   12B                28  .

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\herms\anaconda3\envs\apenv\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\herms\AppData\Local\Temp\ipykernel_3664\1824617209.py", line 2, in close_all_windows
    main_window.destroy()
  File "c:\Users\herms\anaconda3\envs\apenv\Lib\tkinter\__init__.py", line 2368, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed


KeyboardInterrupt: 